## RAG pipeline from Scratch

#### The Goal of RAG is to take information and pass it to an LLM so that it can generate output based on that information 

* Retrieval - Find relavent info from a query
* Augmented - Augment out input to the LLM
* Generation - Generative output from LLM

## STEPS

1. Open a PDF document (even a collection of PDFs)
2. Format the text of the PDF textbook ready for an embedding model.
3. Embed all of the chunks of text in the textbook and turn them into numerical embeddings which can store for later.
4. Build a retrieval system that uses vector search to find relevant chunck of text based ona aquery
5. Create a prompt that incorporates the relevant pieces of text.
6. Generat an answer to a query based on the passages of the textbook with an LLM

In [1]:
# downloading requirements
!pip install PyMuPDF
!pip install tqdm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 30.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.8/15.8 MB 40.9 MB/s eta 0:00:0000:0100:01


In [2]:
# Step 1
import os

# get pdf path
pdf_path = "/kaggle/input/nutrition-rag/human-nutrition-text.pdf"

In [3]:
# Step 2

import fitz
from tqdm.auto import tqdm 

def text_formatter(text: str) ->str:
    cleaned_text = text.replace("\n"," ").strip()
    
    return cleaned_text

def open_and_read_pdf(pdf_path: str) -> list[dict]:
    doc = fitz.open(pdf_path)
    pages_and_texts = []
    
    for page_number, page in tqdm(enumerate(doc)):
#         print(page_number)
        text = page.get_text()
        text = text_formatter(text=text)
        pages_and_texts.append({"page_number": page_number - 41,
                               "page_char_count": len(text),
                               "page_word_count": len(text.split(" ")),
                               "page_sentence_count_raw": len(text.split(".")),
                               "page_token_count": len(text)/4, # 1 toke  is ~4 characters
                               "text":text})
        
    return pages_and_texts
    
pages_and_texts = open_and_read_pdf(pdf_path=pdf_path)
pages_and_texts[:2]

0it [00:00, ?it/s]

[{'page_number': -41,
  'page_char_count': 29,
  'page_word_count': 4,
  'page_sentence_count_raw': 1,
  'page_token_count': 7.25,
  'text': 'Human Nutrition: 2020 Edition'},
 {'page_number': -40,
  'page_char_count': 0,
  'page_word_count': 1,
  'page_sentence_count_raw': 1,
  'page_token_count': 0.0,
  'text': ''}]

In [4]:
import pandas as pd

df = pd.DataFrame(pages_and_texts)
df.head()

,page_number,page_char_count,page_word_count,page_sentence_count_raw,page_token_count,text
0,-41,29,4,1,7.25,Human Nutrition: 2020 Edition
1,-40,0,1,1,0.00,
2,-39,320,54,1,80.00,Human Nutrition: 2020 Edition UNIVERSITY OF ...
3,-38,212,32,3,53.00,Human Nutrition: 2020 Edition by University of...
4,-37,797,145,3,199.25,Contents Preface University of Hawai‘i at Mā...


In [5]:
df.describe()

,page_number,page_char_count,page_word_count,page_sentence_count_raw,page_token_count
count,1208.00000,1208.000000,1208.000000,1208.000000,1208.000000
mean,562.50000,1148.594371,198.889901,14.180464,287.148593
std,348.86387,560.441673,95.747365,9.544587,140.110418
min,-41.00000,0.000000,1.000000,1.000000,0.000000
25%,260.75000,762.750000,134.000000,8.000000,190.687500
50%,562.50000,1232.500000,215.000000,13.000000,308.125000
75%,864.25000,1605.250000,271.250000,19.000000,401.312500
max,1166.00000,2308.000000,429.000000,82.000000,577.000000


In [6]:
# Splitting text in page into sentences, using NLP lib - spaCy
from spacy.lang.en import English

nlp = English()
nlp.add_pipe("sentencizer")

for item in tqdm(pages_and_texts):
    item["sentences"] = list(nlp(item['text']).sents)
    
    # Make sure all sentences are strings
    item["sentences"] = [str(sentence) for sentence in item["sentences"]]
    
    #Count the sentences
    item["page_sentence_count_spacy"] = len(item["sentences"])

  0%|          | 0/1208 [00:00<?, ?it/s]

In [7]:
import random

random.sample(pages_and_texts, k=1)

[{'page_number': 80,
  'page_char_count': 1169,
  'page_word_count': 196,
  'page_sentence_count_raw': 9,
  'page_token_count': 292.25,
  'text': 'All eleven organ systems in the human body require nutrient input  to perform their specific biological functions. Overall health and the  ability to carry out all of life’s basic processes is fueled by energy- supplying nutrients (carbohydrate, fat, and protein). Without them,  organ systems would fail, humans would not reproduce, and the  race would disappear. In this section, we will discuss some of the  critical nutrients that support specific organ system functions.  Learning Activities  Technology Note: The second edition of the Human  Nutrition Open Educational Resource (OER) textbook  features interactive learning activities.\xa0 These activities are  available in the web-based textbook and not available in the  downloadable versions (EPUB, Digital PDF, Print_PDF, or  Open Document).  Learning activities may be used across various mo

In [8]:
# STEP 3

# Chucking our sentences together
# Chucking - grouping sentences into groups

# We do this because, to make our text chunks fit into our embedding model context window

num_sentence_chunk_size = 10

# eg: [25] => [10,10,5] 

def split_list(input_list:list[str],
              slice_size: int=num_sentence_chunk_size) -> list[list[str]]:
    return [input_list[i:i+slice_size] for i in range(0, len(input_list),slice_size)]

# Testing
test_list = list(range(25))
split_list(test_list)

[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
 [10, 11, 12, 13, 14, 15, 16, 17, 18, 19],
 [20, 21, 22, 23, 24]]

In [9]:
# Loop through pages and texts and split sentences into chunks
for item in tqdm(pages_and_texts):
    item["sentence_chunks"] = split_list(input_list = item["sentences"],
                                        slice_size=num_sentence_chunk_size)
    item["num_chunks"] = len(item["sentence_chunks"])

  0%|          | 0/1208 [00:00<?, ?it/s]

In [10]:
random.sample(pages_and_texts, k=1)

[{'page_number': 712,
  'page_char_count': 1834,
  'page_word_count': 340,
  'page_sentence_count_raw': 19,
  'page_token_count': 458.5,
  'text': 'EAR values become the scientific foundation upon which RDA  values are set.  2. Recommended Daily Allowances. Once the EAR of a nutrient  has been established, the RDA can be mathematically  determined. While the EAR is set at a point that meets the  needs of half the population, RDA values are set to meet the  needs of the vast majority (97 to 98 percent) of the target  healthy population. It is important to note that RDAs are not  the same thing as individual nutritional requirements. The  actual nutrient needs of a given individual will be different  than the RDA. However, since we know that 97 to 98 percent of  the population’s needs are met by the RDA, we can assume that  if a person is consuming the RDA of a given nutrient, they are  most likely meeting their nutritional need for that nutrient.  The important thing to remember is that

In [11]:
# Split each chunk into its own item
import re

# Split each chunk into its own item
pages_and_chunks = []
for item in tqdm(pages_and_texts):
    for sentence_chunk in item["sentence_chunks"]:
        chunk_dict = {}
        chunk_dict["page_number"] = item["page_number"]
        
        # Join the sentences together into a paragraph-like structure, aka a chunk (so they are a single string)
        joined_sentence_chunk = "".join(sentence_chunk).replace("  ", " ").strip()
        joined_sentence_chunk = re.sub(r'\.([A-Z])', r'. \1', joined_sentence_chunk) # ".A" -> ". A" for any full-stop/capital letter combo 
        chunk_dict["sentence_chunk"] = joined_sentence_chunk

        # Get stats about the chunk
        chunk_dict["chunk_char_count"] = len(joined_sentence_chunk)
        chunk_dict["chunk_word_count"] = len([word for word in joined_sentence_chunk.split(" ")])
        chunk_dict["chunk_token_count"] = len(joined_sentence_chunk) / 4 # 1 token = ~4 characters
        
        pages_and_chunks.append(chunk_dict)

  0%|          | 0/1208 [00:00<?, ?it/s]

In [12]:
# View a random sample
random.sample(pages_and_chunks, k=1)

[{'page_number': -35,
  'sentence_chunk': 'The Cardiovascular System University of Hawai‘i at Mānoa Food Science and Human Nutrition Program and Human Nutrition Program 82 Central Nervous System University of Hawai‘i at Mānoa Food Science and Human Nutrition Program and Human Nutrition Program 94 The Respiratory System University of Hawai‘i at Mānoa Food Science and Human Nutrition Program and Human Nutrition Program 99 The Endocrine System University of Hawai‘i at Mānoa Food Science and Human Nutrition Program and Human Nutrition Program 106 The Urinary System University of Hawai‘i at Mānoa Food Science and Human Nutrition Program and Human Nutrition Program 110 The Muscular System University of Hawai‘i at Mānoa Food Science and Human Nutrition Program and Human Nutrition Program 117 The Skeletal System University of Hawai‘i at Mānoa Food Science and Human Nutrition Program and Human Nutrition Program 120 The Immune System University of Hawai‘i at Mānoa Food Science and Human Nutritio

In [13]:
df = pd.DataFrame(pages_and_chunks)
df.describe().round(2)

,page_number,chunk_char_count,chunk_word_count,chunk_token_count
count,1843.00,1843.00,1843.00,1843.00
mean,583.38,734.83,112.72,183.71
std,347.79,447.43,71.07,111.86
min,-41.00,12.00,3.00,3.00
25%,280.50,315.00,45.00,78.75
50%,586.00,746.00,114.00,186.50
75%,890.00,1118.50,173.00,279.62
max,1166.00,1831.00,297.00,457.75


In [14]:
# Show random chunks with under 30 tokens in length
min_token_length = 30
for row in df[df["chunk_token_count"] <= min_token_length].sample(5).iterrows():
    print(f'Chunk token count: {row[1]["chunk_token_count"]} | Text: {row[1]["sentence_chunk"]}')

Chunk token count: 26.25 | Text: Updated November 6, 2015. Accessed April 15, 2018. 1122 | Undernutrition, Overnutrition, and Malnutrition
Chunk token count: 11.25 | Text: Accessed March 17, 2011. 212 | Water Concerns
Chunk token count: 16.0 | Text: PART II CHAPTER 2. THE HUMAN BODY Chapter 2. The Human Body | 53
Chunk token count: 11.0 | Text: 978 | Food Supplements and Food Replacements
Chunk token count: 19.5 | Text: http:/ /pressbooks.oer.hawaii.edu/ humannutrition2/?p=519   Introduction | 991


In [15]:
# filter our DataFrame for rows with under 30 tokens

pages_and_chunks_over_min_token_len = df[df["chunk_token_count"] > min_token_length].to_dict(orient="records")
pages_and_chunks_over_min_token_len[:2]

[{'page_number': -39,
  'sentence_chunk': 'Human Nutrition: 2020 Edition UNIVERSITY OF HAWAI‘I AT MĀNOA FOOD SCIENCE AND HUMAN NUTRITION PROGRAM ALAN TITCHENAL, SKYLAR HARA, NOEMI ARCEO CAACBAY, WILLIAM MEINKE-LAU, YA-YUN YANG, MARIE KAINOA FIALKOWSKI REVILLA, JENNIFER DRAPER, GEMADY LANGFELDER, CHERYL GIBBY, CHYNA NICOLE CHUN, AND ALLISON CALABRESE',
  'chunk_char_count': 308,
  'chunk_word_count': 42,
  'chunk_token_count': 77.0},
 {'page_number': -38,
  'sentence_chunk': 'Human Nutrition: 2020 Edition by University of Hawai‘i at Mānoa Food Science and Human Nutrition Program is licensed under a Creative Commons Attribution 4.0 International License, except where otherwise noted.',
  'chunk_char_count': 210,
  'chunk_word_count': 30,
  'chunk_token_count': 52.5}]

In [16]:
# Embedding our text chunks

# using all-mpnet-base-v2 sentence transformer

!pip install -U sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 4.0 MB/s eta 0:00:00a 0:00:01


In [17]:
from sentence_transformers import SentenceTransformer
sentences = ["This is an example sentence", "Each sentence is converted"]

embedding_model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2',
                           device = "cuda")

2024-06-21 14:39:54.518828: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-21 14:39:54.518929: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-21 14:39:54.647796: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [18]:
# Turn text chunks into a single list
text_chunks = [item["sentence_chunk"] for item in pages_and_chunks_over_min_token_len]

In [19]:
%%time

# Embed all texts in batches
text_chunk_embeddings = embedding_model.encode(text_chunks,
                                               batch_size=32,
                                               convert_to_tensor=True) # optional to return embeddings as tensor instead of array

text_chunk_embeddings

Batches:   0%|          | 0/53 [00:00<?, ?it/s]

CPU times: user 15.3 s, sys: 131 ms, total: 15.4 s
Wall time: 13.8 s


tensor([[ 0.0674,  0.0902, -0.0051,  ..., -0.0221, -0.0232,  0.0126],
        [ 0.0552,  0.0592, -0.0166,  ..., -0.0120, -0.0103,  0.0227],
        [ 0.0280,  0.0340, -0.0206,  ..., -0.0054,  0.0213,  0.0313],
        ...,
        [ 0.0771,  0.0098, -0.0122,  ..., -0.0409, -0.0752, -0.0241],
        [ 0.1030, -0.0165,  0.0083,  ..., -0.0574, -0.0283, -0.0295],
        [ 0.0864, -0.0125, -0.0113,  ..., -0.0522, -0.0337, -0.0299]],
       device='cuda:0')

In [20]:
# Save embeddings to file
text_chunks_and_embeddings_df = pd.DataFrame(pages_and_chunks_over_min_token_len)
embeddings_df_save_path = "text_chunks_and_embeddings_df.csv"
text_chunks_and_embeddings_df.to_csv(embeddings_df_save_path, index=False)

In [21]:
# Import saved file and view
text_chunks_and_embedding_df_load = pd.read_csv(embeddings_df_save_path)
text_chunks_and_embedding_df_load.head()

,page_number,sentence_chunk,chunk_char_count,chunk_word_count,chunk_token_count
0,-39,Human Nutrition: 2020 Edition UNIVERSITY OF HA...,308,42,77.00
1,-38,Human Nutrition: 2020 Edition by University of...,210,30,52.50
2,-37,Contents Preface University of Hawai‘i at Māno...,766,114,191.50
3,-36,Lifestyles and Nutrition University of Hawai‘i...,941,142,235.25
4,-35,The Cardiovascular System University of Hawai‘...,998,152,249.50
